# Fooling LIME and SHAP
## Hyperparameter Sensitivity (10)

Analyze the impact of the hyperparameters of LIME and SHAP (e.g., hyperparameters of the local model and of the pertubation algorithms).

In [1]:
import os

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from fooling_LIME_SHAP_Interface.adversarial_model_toolbox import AdversarialModelToolbox
from fooling_LIME_SHAP_Interface.perturbator import Perturbator
from fooling_LIME_SHAP_Interface.util.explainer_type import ExplainerType
from fooling_LIME_SHAP_Interface.util.ml_type import MLType
from fooling_LIME_SHAP_Interface.util.pertubation_method import PerturbationMethod

First we define all the neccessary variables.

In [2]:
TRAIN_TEST_SPLIT = 0.2
SEED = 666

dataset_name = 'HousingData'
input_ids = [5, 7, 11, 12]
categorical_input_ids = []
output_id = 13
biased_id = 11  # B

categorical_input_ids = [input_ids.index(cat_id) for cat_id in categorical_input_ids]
biased_id = input_ids.index(biased_id)

Loading the dataset and doing some preprocessing. There are two datasets. One Dataset is with the biased feature and the other dataset is unbiased because all rows from the biased feature are set to 0.

In [3]:
# Load dataset
house_df = pd.read_csv(os.path.join("datasets", dataset_name + ".csv"))
house_df = house_df.fillna(0)

# Split input and output
y_df = house_df.iloc[:, output_id]
x_df = house_df.iloc[:, input_ids]

scaler = MinMaxScaler()
scaler.fit(x_df.to_numpy())
x = scaler.transform(x_df.to_numpy())

# Create train and test data
x_train, x_test, y_train, y_test = train_test_split(x, y_df.to_numpy(), test_size=TRAIN_TEST_SPLIT, random_state=SEED)

# Create training data without biased column
ux_train = x_train.copy()
ux_train[:, biased_id] = 0
ux_test = x_test.copy()
ux_test[:, biased_id] = 0

Train the biased and unbiased ML model

In [4]:
biased_ml = RandomForestRegressor(random_state=SEED)
biased_ml.fit(x_train, y_train)
print("Accuracy of biased model: {0:3.2}".format(biased_ml.score(x_test, y_test)))

Accuracy of biased model: 0.77


In [5]:
unbiased_ml = RandomForestRegressor(random_state=SEED)
unbiased_ml.fit(ux_test, y_test)
print("Accuracy of unbiased model: {0:3.2}".format(unbiased_ml.score(ux_test, y_test)))

Accuracy of unbiased model: 0.96


## Hyperparameter sensitivity local model

In [6]:
for hp in [1, 10, 100, 200]:
    print("============================== rf_estimators: {} ==============================".format(hp))
    adv = AdversarialModelToolbox(biased_model=biased_ml, 
                                  x_train=x_train, 
                                  y_train=y_train, 
                                  x_test=x_test, 
                                  y_test=y_test,
                                  input_feature_names=house_df.columns[input_ids].tolist(),
                                  categorical_feature_indices=categorical_input_ids,
                                  unbiased_model=unbiased_ml,
                                  biased_id=biased_id, 
                                  fool_explainer_type=ExplainerType.LIME,
                                  ml_type=MLType.REGRESSION, 
                                  seed=SEED)
    adv.train(rf_estimators=hp, perturbator=Perturbator(PerturbationMethod.LIME, perturbation_multiplier=30, perturbation_std=0.3, seed=SEED))
    adv.get_explanations(explanation_sample_number=-1)

============================== rf_estimators: 1 ==============================
Calculating Lime explanations
Original Lime explanation:
{'RM': 4.222782069571837, 'LSTAT': -3.4068982139961905, 'DIS': -1.9202133425684937, 'B': 0.3780400306591363}
Adversarial Lime explanation:
{'LSTAT': -4.988424142473841, 'RM': 2.300545566293219, 'DIS': -1.3530006798242906, 'B': 0.009684621136258658}
Prediction fidelity between original and adversarial model: 0.66
============================== rf_estimators: 10 ==============================
Calculating Lime explanations
Original Lime explanation:
{'RM': 4.222782069571837, 'LSTAT': -3.4068982139961905, 'DIS': -1.9202133425684937, 'B': 0.3780400306591363}
Adversarial Lime explanation:
{'LSTAT': -5.005607314849257, 'RM': 2.271766558393681, 'DIS': -1.3613040521556339, 'B': 0.01247091836673816}
Prediction fidelity between original and adversarial model: 0.75
============================== rf_estimators: 100 ==============================
Calculating Lime ex

## Hyperparameter sensitivity pertubation multiplier

In [7]:
for hp in [1, 10, 20, 30]:
    print("============================== perturbation_multiplier: {} ==============================".format(hp))
    adv = AdversarialModelToolbox(biased_model=biased_ml, 
                                  x_train=x_train, 
                                  y_train=y_train, 
                                  x_test=x_test, 
                                  y_test=y_test,
                                  input_feature_names=house_df.columns[input_ids].tolist(),
                                  categorical_feature_indices=categorical_input_ids,
                                  unbiased_model=unbiased_ml,
                                  biased_id=biased_id, 
                                  fool_explainer_type=ExplainerType.LIME,
                                  ml_type=MLType.REGRESSION, 
                                  seed=SEED)
    adv.train(rf_estimators=100, perturbator=Perturbator(PerturbationMethod.LIME, perturbation_multiplier=hp, perturbation_std=0.3, seed=SEED))
    adv.get_explanations(explanation_sample_number=-1)

============================== perturbation_multiplier: 1 ==============================
Calculating Lime explanations
Original Lime explanation:
{'RM': 4.222782069571837, 'LSTAT': -3.4068982139961905, 'DIS': -1.9202133425684937, 'B': 0.3780400306591363}
Adversarial Lime explanation:
{'LSTAT': -4.7772266626494195, 'RM': 2.635345906426768, 'DIS': -1.4979112780321497, 'B': -0.06567992222295048}
Prediction fidelity between original and adversarial model: 0.95
============================== perturbation_multiplier: 10 ==============================
Calculating Lime explanations
Original Lime explanation:
{'RM': 4.222782069571837, 'LSTAT': -3.4068982139961905, 'DIS': -1.9202133425684937, 'B': 0.3780400306591363}
Adversarial Lime explanation:
{'LSTAT': -4.942870750251516, 'RM': 2.3311859270326725, 'DIS': -1.3939305048627344, 'B': 0.018870048419982376}
Prediction fidelity between original and adversarial model: 0.84
============================== perturbation_multiplier: 20 ==================

## Hyperparameter sensitivity pertubation std

In [8]:
for hp in [0.1, 0.25, 0.5, 0.75, 1.0]:
    print("============================== perturbation_std: {} ==============================".format(hp))

    adv = AdversarialModelToolbox(biased_model=biased_ml, 
                                  x_train=x_train, 
                                  y_train=y_train, 
                                  x_test=x_test, 
                                  y_test=y_test,
                                  input_feature_names=house_df.columns[input_ids].tolist(),
                                  categorical_feature_indices=categorical_input_ids,
                                  unbiased_model=unbiased_ml,
                                  biased_id=biased_id, 
                                  fool_explainer_type=ExplainerType.LIME,
                                  ml_type=MLType.REGRESSION, 
                                  seed=SEED)
    adv.train(rf_estimators=100, perturbator=Perturbator(PerturbationMethod.LIME, perturbation_multiplier=30, perturbation_std=hp, seed=SEED))
    adv.get_explanations(explanation_sample_number=-1)

============================== perturbation_std: 0.1 ==============================
Calculating Lime explanations
Original Lime explanation:
{'RM': 4.222782069571837, 'LSTAT': -3.4068982139961905, 'DIS': -1.9202133425684937, 'B': 0.3780400306591363}
Adversarial Lime explanation:
{'LSTAT': -5.052275334226851, 'RM': 2.249697653631278, 'DIS': -1.3394980896802413, 'B': -0.002337301715344674}
Prediction fidelity between original and adversarial model: 0.42
============================== perturbation_std: 0.25 ==============================
Calculating Lime explanations
Original Lime explanation:
{'RM': 4.222782069571837, 'LSTAT': -3.4068982139961905, 'DIS': -1.9202133425684937, 'B': 0.3780400306591363}
Adversarial Lime explanation:
{'LSTAT': -5.034743745022523, 'RM': 2.25546621306218, 'DIS': -1.3439919224267058, 'B': 0.00954131099902666}
Prediction fidelity between original and adversarial model: 0.67
============================== perturbation_std: 0.5 ==============================
Calcul